# **2.3 Elimination Using Matrices**

### **Goals of this chapter:**

1. See how each elimination step is actually a **matrix multiplication**.  
2. Assemble all those steps $E_{ij}$ into one elimination matrix $E$.  
3. Understand how each $E_{ij}$ is inverted by its inverse matrix $E^{-1}_{ij}$.  
4. Assemble all those inverses (in the correct order) into the **lower-triangular matrix** $L$.


### **Matrices, Vectors, and $Ax=b$**

Matrix–vector multiplication uses dot products:

$$
(Ax)_i = \sum_{j=1}^n a_{ij} x_j
$$


**The Matrix Form of One Elimination Step**

In matrix elimination, we **literally use a matrix** to eliminate an entry in a vector or another matrix.

Let the vector

$$
b = \begin{bmatrix} 2 \\ 8 \\ 10 \end{bmatrix}
\quad \longrightarrow \quad
b_{\text{new}} = \begin{bmatrix} 2 \\ 4 \\ 10 \end{bmatrix}
$$

We want a matrix that performs  
**“row2 → row2 − 2·row1”**.

The elimination matrix is:

$$
E = \begin{bmatrix}
1 & 0 & 0 \\
-2 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

Multiply:

$$
Eb = 
\begin{bmatrix}
1 & 0 & 0 \\
-2 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix} 2 \\ 8 \\ 10 \end{bmatrix}
=
\begin{bmatrix}
2 \\ 4 \\ 10
\end{bmatrix}
$$

- Rows 1 and 3 stay the same because they match the identity matrix.
- The elimination matrix $E_{ij}$ has one extra nonzero entry $-l$ at position $(i,j)$.
- Its effect: **subtract $l$ times row $j$ from row $i$**.

When we multiply **both sides of $Ax=b$** by $E_{ij}$, the goal is to create a **zero** in the $(i,j)$ position of $A$.


In [35]:
def I(n):
    return [[1 if i == j else 0 for j in range(n)] for i in range(n)]

def matmul(A, B):
    rows, cols = len(A), len(B[0])
    out = [[0]*cols for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            out[i][j] = sum(A[i][k] * B[k][j] for k in range(len(B)))
    return out

I(3)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [26]:
def E_matrix(n, r, pivot, factor):
    E = I(n)
    E[r][pivot] = -factor
    return E

E = E_matrix(3, r=1, pivot=0, factor=4/1)
E

[[1, 0, 0], [-2, 1, 0], [0, 0, 1]]

In [37]:
A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
matmul(E, A)

[[1, 2, 3], [0.0, -3.0, -6.0], [7, 8, 9]]

### **Matrix Multiplication**

To multiply matrices:

1. **Associative:**  
   $$A(BC) = (AB)C$$

2. **Not commutative:**  
   $$AB \neq BA\text{ often}$$

If $B$ has columns $b_1, b_2, b_3$, then:

$$
AB = A[b_1 \ b_2 \ b_3] = [Ab_1 \ Ab_2 \ Ab_3]
$$

Matrix multiplication acts **column-by-column**.


In [4]:
def matrix_multiplication(A, B, trace=False):
    m, n = len(A), len(A[0])
    n2, p = len(B), len(B[0])

    # Shape validation
    if n != n2:
        raise ValueError(f"Invalid shapes")

    # Initialize output matrix C (mxp)
    C = [[0 for _ in range(p)] for _ in range(m)]

    # Compute multiplication
    for i in range(m):
        for j in range(p):
            dot_sum = 0
            if trace:
                print(f"Computing C[{i}][{j}] = dot(row {i}, col {j})")
    
            for k in range(n):
                product = A[i][k] * B[k][j] # product of each entry
                dot_sum += product # dot_product of each row
    
                if trace:
                    print(f"A[{i}][{k}] * B[{i}][{k}] = {A[i][k]} * {B[k][j]} = {product}")
    
            C[i][j] = dot_sum

            if trace:
                print(f"=> C[{i}][{j}] = {dot_sum}\n")

    return C

    
A = [[1, 2, 3], [4, 5, 6]]
B = [[7, 8], [9, 10], [11, 12]]
C = matrix_multiplication(A, B, trace=True)
print("Result")
for row in C:
    print(row)

Computing C[0][0] = dot(row 0, col 0)
A[0][0] * B[0][0] = 1 * 7 = 7
A[0][1] * B[0][1] = 2 * 9 = 18
A[0][2] * B[0][2] = 3 * 11 = 33
=> C[0][0] = 58

Computing C[0][1] = dot(row 0, col 1)
A[0][0] * B[0][0] = 1 * 8 = 8
A[0][1] * B[0][1] = 2 * 10 = 20
A[0][2] * B[0][2] = 3 * 12 = 36
=> C[0][1] = 64

Computing C[1][0] = dot(row 1, col 0)
A[1][0] * B[1][0] = 4 * 7 = 28
A[1][1] * B[1][1] = 5 * 9 = 45
A[1][2] * B[1][2] = 6 * 11 = 66
=> C[1][0] = 139

Computing C[1][1] = dot(row 1, col 1)
A[1][0] * B[1][0] = 4 * 8 = 32
A[1][1] * B[1][1] = 5 * 10 = 50
A[1][2] * B[1][2] = 6 * 12 = 72
=> C[1][1] = 154

Result
[58, 64]
[139, 154]


### **The Matrix $P_{ij}$ for a Row Exchange**

If a pivot is zero, elimination fails temporarily, so we **exchange rows**.

A row exchange is done by a **permutation matrix**.

Exchange rows 2 and 3:

$$
P_{23} = 
\begin{bmatrix}
1 & 0 & 0 \\
0 & 0 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

- It is just the identity matrix with rows 2 and 3 swapped.
- When multiplied with any matrix, it swaps rows 2 and 3 of that matrix.

In [38]:
def P_matrix(n, i, j):
    P = I(n)
    P[i], P[j] = P[j], P[i]
    return P

P = P_matrix(3, 1, 2)
P

[[1, 0, 0], [0, 0, 1], [0, 1, 0]]

In [39]:
matmul(P, A)

[[1, 2, 3], [7, 8, 9], [4, 5, 6]]

### **The Augmented Matrix**

Elimination applies the same row operations to both $A$ and $b$.

So we combine them:

$$
[A \ b] =
\begin{bmatrix}
2 & 4 & -2 & 2 \\
4 & 9 & -3 & 8 \\
-2 & -3 & 7 & 10
\end{bmatrix}
$$

Apply the elimination matrix $E$ to the entire augmented matrix:

$$
E [A \ b] =
\begin{bmatrix}
1 & 0 & 0 \\
-2 & 1 & 0 \\
0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
2 & 4 & -2 & 2 \\
4 & 9 & -3 & 8 \\
-2 & -3 & 7 & 10
\end{bmatrix}
=
\begin{bmatrix}
2 & 4 & -2 & 2 \\
0 & 1 & 1 & 4 \\
-2 & -3 & 7 & 10
\end{bmatrix}
$$

- **Row view:** $E$ acts on each row of $[A\ b]$.  
- **Column view:** $E$ acts on each column of $[A\ b]$ separately.


**Key Ideas**

1.  
   $$
   Ax = x_1(\text{col }1) + \cdots + x_n(\text{col }n),\quad
   (Ax)_i = \sum_{j=1}^n a_{ij} x_j
   $$

2. Identity matrix = $I$.  
   Elimination matrix = $E_{ij}$ using multiplier $l_{ij}$.  
   Exchange matrix = $P_{ij}$.

3. Multiplying $Ax=b$ by $E_{21}$ subtracts $l_{21}$ times equation 1 from equation 2.  
   The number $-l_{21}$ is the $(2,1)$ entry of $E_{21}$.

4. For the augmented matrix $[A\ b]$, the elimination step becomes:

   $$
   E_{21}[A\ b] = [E_{21}A \ \ E_{21}b]
   $$

5. When $A$ multiplies **any** matrix $B$, it multiplies **each column of $B$** independently.
